In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

In [2]:
# build spark session
spark = SparkSession.builder.appName("csv_change_timestamp").getOrCreate()

23/12/05 18:15:43 WARN Utils: Your hostname, neivekim76.local resolves to a loopback address: 127.0.0.1; using 172.30.33.250 instead (on interface en0)
23/12/05 18:15:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/05 18:15:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/05 18:15:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# create rdd
data_dir = "/Users/kimdohoon/git/datas/ml-latest/ratings.csv"
rdd = spark.sparkContext.textFile(data_dir).map(lambda line: line.split(',')) \
                                           .filter(lambda x: all(item != "" for item in x)) \
                                           .filter(lambda x: len(x) == 4)

In [4]:
def is_int_convertible(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

# rdd.filter()
# 함수의 출력이 "논리깂"을 반환하도록 구성
filtered_rdd = rdd.filter(lambda x: is_int_convertible(x[3]))

In [5]:
def convert_timestamp(row):
    from datetime import datetime
    
    timestamp = int(row[3])
    dt_object = datetime.fromtimestamp(timestamp)
    formatted_date = dt_object.strftime('%Y-%m-%d')
    formatted_time = dt_object.strftime('%H:%M:%S')
    return (row[0], row[1], row[2], formatted_date, formatted_time)  # Assuming you have other columns in the CSV

# rdd.map()
# 함수의 출력이 "tuple"을 반환하도록 구성
converted_rdd = filtered_rdd.map(convert_timestamp)

In [6]:
# change rdd to dataframe
column = ['userId','movieId','rating','date','time']
df = converted_rdd.toDF(column)
df.show()

23/12/05 18:16:29 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker


+------+-------+------+----------+--------+
|userId|movieId|rating|      date|    time|
+------+-------+------+----------+--------+
|     1|      1|   4.0|2008-11-04|02:52:19|
|     1|    110|   4.0|2008-11-05|15:04:46|
|     1|    158|   4.0|2008-11-04|02:31:43|
|     1|    260|   4.5|2008-11-04|03:00:04|
|     1|    356|   5.0|2008-11-04|02:58:39|
|     1|    381|   3.5|2008-11-04|02:41:45|
|     1|    596|   4.0|2008-11-04|02:32:04|
|     1|   1036|   5.0|2008-11-04|03:07:06|
|     1|   1049|   3.0|2008-11-04|02:41:19|
|     1|   1066|   4.0|2008-11-04|03:29:21|
|     1|   1196|   3.5|2008-11-04|03:04:01|
|     1|   1200|   3.5|2008-11-04|03:11:01|
|     1|   1210|   4.5|2008-11-04|03:00:10|
|     1|   1214|   4.0|2008-11-04|03:20:26|
|     1|   1291|   5.0|2008-11-04|02:53:29|
|     1|   1293|   2.0|2008-11-04|02:37:22|
|     1|   1376|   3.0|2008-11-04|02:32:19|
|     1|   1396|   3.0|2008-11-04|02:32:14|
|     1|   1537|   4.0|2008-11-04|03:24:47|
|     1|   1909|   3.0|2008-11-0